# ***Extra Credit Assignment 1***

In [1]:
import os
print(os.getcwd())  # Check current working directory
os.chdir('/Users/devynmiller/Downloads/ec1-cpsc542')  # Change to your project directory

/Users/devynmiller/Downloads/ec1-cpsc542


In [2]:
import warnings
warnings.filterwarnings("ignore")
import importlib


In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# **Preprocessing**

### Import augmentation module

In [ ]:
#preprocessing.py
import src.augmentation as augmentation
importlib.reload(augmentation)

# Initialize augmentation
aug = augmentation()

### Load data

In [4]:
#preprocessing.py
import src.dataloader_eda as dataloader_eda
importlib.reload(dataloader_eda)

# Load data
test_path, train_path, train = dataloader_eda.load_data()

# Use the data generator with augmentation
generator = dataloader_eda.data_generator(train, 16, train_path, augmentation=aug)

ModuleNotFoundError: No module named 'augmentation'

### Augmentation and EDA

In [ ]:
# Test the generator
example, label = next(dataloader_eda.data_generator(batch_size=1))
img = example['image'][0]
bbox_coords = label['coords'][0]

In [ ]:
dataloader_eda.display_image_from_file("vid_4_10520.jpg")

dataloader_eda.display_image(img, bbox_coords=bbox_coords, norm=True)
dataloader_eda.display_grid(train)

# **Model**

### Model

In [ ]:
import model as model
importlib.reload(model)

model, test = model.modeling()

### Prediction

In [ ]:
import src.predict as predict
importlib.reload(predict)

image_path = "example_image.jpg"
predicted_bbox = predict.predict_bounding_box(image_path)
print(f"Predicted bounding box for {image_path}: {predicted_bbox}")

### Training

In [ ]:
import src.train as train_model
importlib.reload(train_model)
import json

train.train_model()

# **Validation**

### Loss Metrics/Graphs

In [ ]:
import src.evaluation as evaluate
importlib.reload(evaluate)

for i in range(5):
    model_path = f'/Users/devynmiller/Downloads/ec1-cpsc542/models/best_model_{i}.h5'
    history, best_3, worst_3, display_with_boxes, generate_gradcam = evaluate.evaluate_model(model_path)
    
    evaluate.plot_model_metrics(history, f'Best Model {i}')
    
    print(f"Best 3 for model {i}")
    for index, (iou, image, pred_box, true_box) in enumerate(best_3):
        save_path_best = f'/Users/devynmiller/Downloads/ec1-cpsc542/PNGs/3b3w/best_model_{i}_best_{index}.png'
        display_with_boxes(image, pred_box, true_box, save_path_best)

    print(f"Worst 3 for model {i}")
    for index, (iou, image, pred_box, true_box) in enumerate(worst_3):
        save_path_worst = f'/Users/devynmiller/Downloads/ec1-cpsc542/PNGs/3b3w/best_model_{i}_worst_{index}.png'
        display_with_boxes(image, pred_box, true_box, save_path_worst)

    img = load_img('/Users/devynmiller/Downloads/ec1-cpsc542/your_image.jpg', target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    layer_name = 'conv2d_9'
    gradcam_image = generate_gradcam(model_path, img_array, layer_name)

    plt.imshow(gradcam_image)
    plt.title(f'GradCAM for Best Model {i}')
    plt.show()

    gradcam_save_path = f'/Users/devynmiller/Downloads/ec1-cpsc542/PNGs/gradcam/gradcam_model_{i}.png'
    plt.imsave(gradcam_save_path, gradcam_image)